In [3]:
import gradio as gr
from transformers import pipeline

# Model from the tutorial
MODEL_NAME = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"

clf = pipeline(
    "image-classification",
    model=MODEL_NAME
)

def analyze_plant(image, temp, humidity, soil):
    preds = clf(image)
    top = preds[0]

    label = top["label"]
    score = top["score"]

    alerts = []
    advice = []

    # --- Conditions based on sensors / user input ---
    if soil < 25:
        alerts.append("Low soil moisture")
        advice.append("Recommendation: irrigate / water the plant")

    if humidity > 80:
        alerts.append("High humidity")
        advice.append("Recommendation: improve ventilation (reduce fungal risk)")

    if temp > 40:
        alerts.append("High temperature")
        advice.append("Recommendation: move the plant to a shaded area")

    # --- Color status ("flag") based on image prediction ---
    # Simple rule: if label contains "healthy" => good (green), else bad (red)
    is_bad = ("healthy" not in label.lower())

    status_html = (
        "<div style='padding:10px;border-radius:10px;"
        f"background:{'#ffdddd' if is_bad else '#ddffdd'};"
        f"border:1px solid {'#ff0000' if is_bad else '#00aa00'};"
        "font-weight:700;'>"
        f"{'🔴 Plant status: BAD' if is_bad else '🟢 Plant status: GOOD'}"
        "</div>"
    )

    if not alerts:
        alerts.append("Status looks normal")

    return (
        f"Detected disease: {label} ({score:.2%})",
        status_html,
        "\n".join(alerts),
        "\n".join(advice)
    )

gr.Interface(
    fn=analyze_plant,
    inputs=[
gr.Image(
    type="filepath",
    label="Upload plant image",
    show_label=True,
    sources=["upload"]
),
        gr.Slider(0, 45, value=25, label="Temperature (°C)"),
        gr.Slider(0, 100, value=50, label="Humidity (%)"),
        gr.Slider(0, 100, value=50, label="Soil Moisture (%)"),
    ],
    outputs=[
        gr.Textbox(
          label="Diagnosis",
          placeholder="Plant disease diagnosis will appear here"
        ),
        gr.HTML(label="Status"),
        gr.Textbox(
          label="Alerts",
          lines=5,
          placeholder="Sensor alerts and warnings will appear here"
        ),
        gr.Textbox(
          label="Recommendations",
          lines=5,
          placeholder="Care and treatment recommendations will appear here"
        ),
    ],
    title="Plant Disease Detection from Image",
    description="Upload a plant image and optionally provide temperature, humidity and soil moisture for basic care recommendations.",
    allow_flagging="never"
).launch()


/usr/local/lib/python3.12/dist-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:30: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(
Device set to use cpu
/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://55c0897eaef816da86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
